In [ ]:
%%capture
!pip install datasets

In [ ]:
%%capture
!pip install accelerate -U

In [ ]:
%%capture
!pip install sentence_transformers

In [ ]:
%%capture
!pip install optuna

In [ ]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding
import matplotlib.pyplot as plt
torch.__version__

'2.3.0+cu121'

In [ ]:
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
    EarlyStoppingCallback,
)
from huggingface_hub import HfFolder, notebook_login

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re

# Load the data
file_path = '/content/training.csv'
data = pd.read_csv(file_path)

# Preprocessing
# 1. Remove unnecessary columns
data = data.drop(columns=['Unnamed: 2'])

# 2. Handle missing values
data = data.dropna(subset=['text'])

# 3. Standardize text (lowercase, remove special characters)
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

data['text'] = data['text'].apply(preprocess_text)

In [ ]:
# Normalizing the class names
def normalize_class(class_name):
    class_name = class_name.lower()
    if class_name == 'pi':
        return 'yes'
    return class_name

# Apply normalization to the 'class' column
data['class'] = data['class'].apply(normalize_class)

# Display the updated class counts
normalized_class_counts = data['class'].value_counts()

print(normalized_class_counts)

class
undefined    780
yes          555
no           348
Name: count, dtype: int64


In [ ]:
undefined_df = data[data['class'] == "undefined"]
undefined_df.to_csv('/content/undefined_df.csv', index=False)

In [ ]:
updated_undefined_df = pd.read_csv('/content/updated_undefined_df.csv')

In [ ]:
updated_undefined_df.head(10)

,class,text
0,no,ive had the iphone 8 plus and iphone x and id ...
1,yes,i need some new phone cases iphonex phonecases...
2,no,jailbroke first time on 1131 using mp great jo...
3,no,cactus travel mug etsyme2c8zcsc craftbuzz craf...
4,yes,official discount code oneplus 6t onepluscomit...
5,no,ever since my dad got his iphone x he been act...
6,yes,free iphone x offer is a most imported offer n...
7,yes,free iphonex offer is a most imported offer no...
8,yes,need a wirelesscharger for the car or you stay...
9,no,b is the iphone x aand p must be the google pi...


In [ ]:
updated_undefined_df['class'].value_counts()

class
no     650
yes    130
Name: count, dtype: int64

In [ ]:
pred_no = updated_undefined_df[updated_undefined_df["class"] == "no"]
pred_yes = updated_undefined_df[updated_undefined_df["class"] == "yes"]

# Combine dataframes vertically
combined_df = pd.concat([data[data["class"] != "undefined"], pred_no[0:337], pred_yes], axis=0)

In [ ]:
combined_df["class"].value_counts()

class
yes    685
no     685
Name: count, dtype: int64

In [ ]:
combined_df.rename(columns={'class': 'label'}, inplace=True)
combined_df['label'] = combined_df['label'].apply(lambda x: 1 if x =='yes' else 0)

In [ ]:
dataset = combined_df[['label','text']]

In [ ]:
dataset.head(10)

,label,text
0,1,whats the difference between the iphone x and ...
1,0,siracusa just listened to hypercritical 16 who...
2,1,same iphone x wont be bad twittercomafuaasamst...
4,0,iphone x for 150 do i have dickhead written al...
5,1,this iphone x still very sexy a year later
7,1,if not im using my kidney omo what is ferrari ...
8,1,iphone x s max for sale
9,0,someone once told me that but ive never seen a...
10,1,screenshots from the iphone x look so clapped
12,1,looks like u now use an iphone x can i have th...


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'dataset' is a DataFrame and 'label' is the name of the target column
X = dataset.drop(columns=['label'])
y = dataset['label']

# Perform stratified split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

# Combine X and y back into DataFrames for training and validation sets
train_roberta = X_train.copy()
train_roberta['label'] = y_train

validation_roberta = X_val.copy()
validation_roberta['label'] = y_val

# train_roberta = dataset.sample(frac=0.75, random_state=42)
# validation_roberta = dataset.sample(frac=0.25, random_state=42)

In [ ]:
# Taking 20% of data from both training and validation data for Hyperparameter Tuning

# TRAINING SPLIT
X = train_roberta.drop(columns=['label'])
y = train_roberta['label']

# Perform stratified split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, stratify=y, random_state=42)

train_roberta_20_percent = X_val.copy()
train_roberta_20_percent['label'] = y_val

# VALIDATION SPLIT
X = validation_roberta.drop(columns=['label'])
y = validation_roberta['label']

# Perform stratified split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, stratify=y, random_state=42)

validation_roberta_20_percent = X_val.copy()
validation_roberta_20_percent['label'] = y_val

In [ ]:
train_roberta['label'].value_counts()

label
1    514
0    513
Name: count, dtype: int64

In [ ]:
validation_roberta['label'].value_counts()

label
0    172
1    171
Name: count, dtype: int64

In [ ]:
print(f'ROBERTA Train Size: {train_roberta.shape}')
print(f'ROBERTA Validation Size: {validation_roberta.shape}')
print(f'ROBERTA Train 20 Percent Size: {train_roberta_20_percent.shape}')
print(f'ROBERTA Validation 20 Percent Size: {validation_roberta_20_percent.shape}')

ROBERTA Train Size: (1027, 2)
ROBERTA Validation Size: (343, 2)
ROBERTA Train 20 Percent Size: (206, 2)
ROBERTA Validation 20 Percent Size: (69, 2)


In [ ]:
train = Dataset.from_pandas(train_roberta, preserve_index=False)
validation = Dataset.from_pandas(validation_roberta, preserve_index=False)
train_for_hyperparameter_tuining = Dataset.from_pandas(train_roberta_20_percent, preserve_index=False)
validation_for_hyperparameter_tuining = Dataset.from_pandas(validation_roberta_20_percent, preserve_index=False)

In [ ]:
print(train)
print(validation)
print(train_for_hyperparameter_tuining)
print(validation_for_hyperparameter_tuining)

Dataset({
    features: ['text', 'label'],
    num_rows: 1027
})
Dataset({
    features: ['text', 'label'],
    num_rows: 343
})
Dataset({
    features: ['text', 'label'],
    num_rows: 206
})
Dataset({
    features: ['text', 'label'],
    num_rows: 69
})


In [ ]:
model_id = "roberta-base"
repository_id = "PiGrieco/OpenSesame"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], max_length=512, padding=True, truncation=True)

In [ ]:
tokenized_train_dataset = train.map(tokenize_function, batched=True)
tokenized_test_dataset = validation.map(tokenize_function, batched=True)

tokenized_train_for_hyperparameter_tuining_dataset = train_for_hyperparameter_tuining.map(tokenize_function, batched=True)
tokenized_test_for_hyperparameter_tuining_dataset = validation_for_hyperparameter_tuining.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/1027 [00:00<?, ? examples/s]

Map:   0%|          | 0/343 [00:00<?, ? examples/s]

Map:   0%|          | 0/206 [00:00<?, ? examples/s]

Map:   0%|          | 0/69 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1027
})

In [ ]:
tokenized_test_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 343
})

In [ ]:
tokenized_train_for_hyperparameter_tuining_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 206
})

In [ ]:
tokenized_test_for_hyperparameter_tuining_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 69
})

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
repository_id = "PiGrieco/Llama3-q4_k_m"
TOKEN ="XXXXXXXXXXXXXXXX"

In [ ]:
notebook_login()

In [ ]:
# Model
model = RobertaForSequenceClassification.from_pretrained(model_id,num_labels=2)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.41.1', '0.30.1')

In [ ]:
import optuna

# Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 4e-5, 6e-5)
    weight_decay = trial.suggest_uniform('weight_decay', 0.008, 0.03)
    num_train_epochs = trial.suggest_int('num_train_epochs', 4, 10)

    # Define the compute_metrics function
    def compute_metrics(eval_preds):
        metric = load_metric("glue", "mrpc")
        logits, labels = eval_preds
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    # TrainingArguments
    training_args = TrainingArguments(
        output_dir=repository_id,
        num_train_epochs=num_train_epochs,
        eval_strategy="epoch",
        logging_dir=f"{repository_id}/logs",
        logging_strategy="steps",
        logging_steps=10,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        save_strategy="epoch",
        load_best_model_at_end=True,
        save_total_limit=2,
        report_to="tensorboard",
        push_to_hub=True,
        hub_strategy="every_save",
        hub_model_id=repository_id,
        hub_token=HfFolder.get_token(),
    )

    model = RobertaForSequenceClassification.from_pretrained(model_id, num_labels=2)

    # Initialize EarlyStoppingCallback
    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=3,  # Number of evaluation steps with no improvement
        early_stopping_threshold=0.0  # Minimum improvement to be considered
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_for_hyperparameter_tuining_dataset,
        eval_dataset=tokenized_test_for_hyperparameter_tuining_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping_callback]  # Add early stopping callback
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_result = trainer.evaluate()
    return eval_result['eval_loss']

# Create Optuna study and optimize
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Print the best trial
print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-06-03 18:46:49,424] A new study created in memory with name: no-name-eb59e3cd-7306-4f45-b339-7c92c5a1de5f
<ipython-input-64-478ed21b2574>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 4e-5, 6e-5)
<ipython-input-64-478ed21b2574>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.008, 0.03)
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this mod

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.667300,0.630982,0.623188,0.711111
2,0.541000,0.537231,0.753623,0.773333
3,0.409500,0.704439,0.739130,0.775000
4,0.116600,0.908817,0.739130,0.742857
5,0.079500,0.997437,0.782609,0.788732


<ipython-input-64-478ed21b2574>:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major rele

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
[I 2024-06-03 18:49:00,209] Trial 0 finished with value: 0.5372306704521179 and parameters: {'learning_rate': 5.7689970933522434e-05, 'weight_decay': 0.012357500718512672, 'num_train_epochs': 5}. Best is trial 0 with value: 0.5372306704521179.
<ipython-input-64-478ed21b2574>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead

...

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
[I 2024-06-03 19:58:18,622] Trial 29 finished with value: 0.6159608364105225 and parameters: {'learning_rate': 5.8222465307701616e-05, 'weight_decay': 0.013340686470563854, 'num_train_epochs': 4}. Best is trial 19 with value: 0.4032965898513794.
<ipython-input-64-478ed21b2574>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) inste

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.692500,0.696825,0.507246,0.000000
2,0.613500,0.553072,0.739130,0.750000
3,0.356500,0.821027,0.695652,0.746988
4,0.329700,1.148015,0.695652,0.752941
5,0.262600,1.106173,0.739130,0.718750


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major rele

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
[I 2024-06-03 20:02:10,294] Trial 30 finished with value: 0.5530720949172974 and parameters: {'learning_rate': 4.602334724531562e-05, 'weight_decay': 0.010970774559016773, 'num_train_epochs': 7}. Best is trial 19 with value: 0.4032965898513794.
<ipython-input-64-478ed21b2574>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instea

Epoch,Training Loss,Validation Loss


In [ ]:
# TrainingArguments
training_args = TrainingArguments(
    output_dir=repository_id,
    num_train_epochs=7,
    eval_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=4.377168455890629e-05,
    weight_decay=0.013354426476286457,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# # TrainingArguments
# training_args = TrainingArguments(
#     output_dir=repository_id,
#     num_train_epochs=trial.params['num_train_epochs'],
#     eval_strategy="epoch",
#     logging_dir=f"{repository_id}/logs",
#     logging_strategy="steps",
#     logging_steps=10,
#     learning_rate=trial.params['learning_rate'],
#     weight_decay=trial.params['weight_decay'],
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     save_total_limit=2,
#     report_to="tensorboard",
#     push_to_hub=True,
#     hub_strategy="every_save",
#     hub_model_id=repository_id,
#     hub_token=HfFolder.get_token(),
# )

In [ ]:
def compute_metrics(eval_preds):   # compute accuracy and f1-score
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

model = RobertaForSequenceClassification.from_pretrained(model_id,num_labels=2)

trainer = Trainer(   # specifying trainer class
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.369700,0.128280,0.970760,0.970930
2,0.057500,0.192709,0.964912,0.966102
3,0.108600,0.083969,0.982456,0.982558
4,0.081000,0.074502,0.985380,0.985507
5,0.251700,0.054279,0.988304,0.988372
6,0.097300,0.053087,0.988304,0.988372
7,0.122900,0.052258,0.988304,0.988372


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major rele

TrainOutput(global_step=903, training_loss=0.11606137541556939, metrics={'train_runtime': 248.4821, 'train_samples_per_second': 28.96, 'train_steps_per_second': 3.634, 'total_flos': 340210816800960.0, 'train_loss': 0.11606137541556939, 'epoch': 7.0})